In [1]:
import pathlib
import cv2

animations_folder = pathlib.Path("animations")
sprites_image = pathlib.Path("NES - Ms Pac-Man Namco - General Sprites.png")
SPRITE_SIZE = 16


def remove_background(sprite: cv2.Mat) -> cv2.Mat:
    tmp = cv2.cvtColor(sprite, cv2.COLOR_BGR2GRAY)
    _,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
    b, g, r = cv2.split(sprite)
    rgba = [b,g,r, alpha]
    return cv2.merge(rgba,4)


def slice_into_sprites(
    image: cv2.Mat, lu_corner: tuple[int, int], shape: tuple[int, int], out_size: int = 32
) -> list[cv2.Mat]:
    result = []
    x, y = lu_corner
    w, h = shape
    
    sprites = image[y:y+h, x:x+w]
    for y_index in range(h // SPRITE_SIZE):
        for x_index in range(w // SPRITE_SIZE):
            sprite = sprites[
                SPRITE_SIZE * y_index:SPRITE_SIZE * (y_index + 1),
                SPRITE_SIZE * x_index:SPRITE_SIZE * (x_index + 1)
            ]
            sprite = remove_background(sprite=sprite)
            result.append(
                cv2.resize(
                    sprite, dsize=(out_size, out_size)
                )
            )

    return result

In [12]:
sprites = slice_into_sprites(
    image=cv2.imread(sprites_image.as_posix(), cv2.IMREAD_UNCHANGED),
    lu_corner=(454, 451),
    shape=(32, 48)
)

for index, sprite in enumerate(sprites):
    cv2.imwrite(
        (animations_folder / f"{index}.png").as_posix(),
        sprite
    )

In [9]:
pacman_sprites = pathlib.Path("s_pacman")
print(pacman_sprites)

for image_path in pacman_sprites.glob("*.png"):
    image = cv2.imread(image_path.as_posix(), cv2.IMREAD_UNCHANGED)
    sprite = remove_background(image)
    
    new_file = pacman_sprites / f"{image_path.stem}_transparent.png"
    cv2.imwrite(new_file.as_posix(), sprite)  

s_pacman
